In [110]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import os
import time
import io
import pyathena as cz
from pyathena.pandas_cursor import PandasCursor
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_similarity_score as jss # evaluation metric
from sklearn.metrics import precision_score # evaluation metric
from sklearn.metrics import classification_report # evaluation metric
from sklearn.metrics import confusion_matrix # evaluation metric
from sklearn.metrics import log_loss # evaluation metric
import pickle

In [102]:
# s3 = boto3.resource('s3')
athena_conection = "athena.us-east-1.amazonaws.com"
region_name = ('us-east-1')
db = "ml_platform_events"

In [103]:
athena_client   = boto3.client('athena', region_name='us-east-1', aws_access_key_id=os.environ['ACCESS_KEY'],
                            aws_secret_access_key=os.environ['ACCESS_SECRET_KEY'])

s3_resource   = boto3.resource('s3', region_name='us-east-1', aws_access_key_id=os.environ['ACCESS_KEY'],
                            aws_secret_access_key=os.environ['ACCESS_SECRET_KEY'])

In [81]:
df = athena_query_to_dataframe(db, "select * from ml_platform_events.cleaned_punkapi")

Starting Query Execution:
Query Succeeded!
Processing Response
Creating Dataframe


KeyError: 'VarCharValue'

In [104]:
df.head()

,id,name,abv,ibu,target_fg,target_og,ebc,srm,ph
0,120,AB:10,11.5,80.0,1020,1096,115.0,57.5,4.4
1,325,Zipcode,5.0,37.0,1006,1044,5.0,3.0,4.4
2,239,#Mashtag 2016,10.5,100.0,1010,1090,15.0,8.0,4.4
3,48,Goldings - IPA Is Dead,6.7,70.0,1015,1066,30.0,15.0,4.4
4,309,Paradox Islay 2018,13.5,100.0,1015,1112,300.0,152.0,4.4


In [108]:
def preditc_classfication():
    model = LogisticRegression()
    
    X = df[['abv', 'target_fg', 'target_og', 'ebc', 'srm', 'ph']]
    y = df['ibu']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    
    model.fit(X_train, y_train)
    
    lr_predict = logisticRegr.predict(X_test)
    lr_proba = logisticRegr.predict_proba(X_test)
    
    print('Jaccard Similarity Score of our model is {}'.format(jss(y_test, lr_predict).round(2)))
    
    ## save on local
    filename = 'model_punkbeer.sav'
    pickle.dump(model, open(filename, 'wb'))
    
    return lr_predict

In [111]:
p = preditc_classfication()
print(p)

Jaccard Similarity Score of our model is 0.5
['70.0' '85.0' '65.0' '80.0']


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
